# Lab 1


## Part 1: Bilingual dictionary induction and unsupervised embedding-based MT (30%)
*Note: this homework is based on materials from yandexdataschool [NLP course](https://github.com/yandexdataschool/nlp_course/). Feel free to check this awesome course if you wish to dig deeper.*

*Refined by [Nikolay Karpachev](https://www.linkedin.com/in/nikolay-karpachev-b0146a104/)*

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully), in particular for similar languages, e.g. Ukrainian and Russian. 

### Frament of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our for our purposes.

## Data

In [0]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

In this notebook we're going to use pretrained word vectors - FastText (original paper - https://arxiv.org/abs/1607.04606).

You can download them from the official [website](https://fasttext.cc/docs/en/crawl-vectors.html). We're going to need embeddings for Russian and Ukrainian languages. Please use word2vec-compatible format (.text).

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.vec.gz

--2019-10-21 23:14:39--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.22.166, 104.20.6.166, 2606:4700:10::6814:6a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.22.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1306357571 (1.2G) [binary/octet-stream]
Saving to: ‘cc.ru.300.vec.gz’

cc.ru.300.vec.gz    100%[===================>]   1.22G  20.3MB/s    in 54s     

2019-10-21 23:15:33 (23.2 MB/s) - ‘cc.ru.300.vec.gz’ saved [1306357571/1306357571]



In [0]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.uk.300.vec.gz

--2019-10-21 23:15:52--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.uk.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.22.166, 104.20.6.166, 2606:4700:10::6814:6a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.22.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1257595219 (1.2G) [binary/octet-stream]
Saving to: ‘cc.uk.300.vec.gz’

cc.uk.300.vec.gz    100%[===================>]   1.17G  23.9MB/s    in 51s     

2019-10-21 23:16:44 (23.3 MB/s) - ‘cc.uk.300.vec.gz’ saved [1257595219/1257595219]



In [0]:
!gunzip cc.ru.300.vec.gz

In [0]:
!gunzip cc.uk.300.vec.gz

In [0]:
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('август', 1.0),
 ('июль', 0.9383153915405273),
 ('сентябрь', 0.9240028858184814),
 ('июнь', 0.9222575426101685),
 ('октябрь', 0.9095538854598999),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545796275138855),
 ('февраль', 0.8401416540145874)]

In [0]:
uk_emb.most_similar([uk_emb["серпень"]])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('серпень', 0.9999999403953552),
 ('липень', 0.9096440076828003),
 ('вересень', 0.901697039604187),
 ('червень', 0.8992519378662109),
 ('жовтень', 0.8810408711433411),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.8408110737800598),
 ('лютий', 0.8256431818008423)]

In [0]:
ru_emb.most_similar([uk_emb["серпень"]])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('Stepashka.com', 0.2757962942123413),
 ('ЖИЗНИВадим', 0.25203436613082886),
 ('2Дмитрий', 0.25048112869262695),
 ('2012Дмитрий', 0.24829231202602386),
 ('Ведущий-Алексей', 0.2443869560956955),
 ('Недопустимость', 0.24435284733772278),
 ('2Михаил', 0.23981399834156036),
 ('лексей', 0.23740756511688232),
 ('комплексн', 0.23695150017738342),
 ('персональ', 0.2368222028017044)]

Load small dictionaries for correspoinding words pairs as trainset and testset.

In [0]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [0]:
!wget -O ukr_rus.train.txt http://tiny.cc/jfgecz

--2019-10-21 23:36:26--  http://tiny.cc/jfgecz
Resolving tiny.cc (tiny.cc)... 192.241.240.89
Connecting to tiny.cc (tiny.cc)|192.241.240.89|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://tiny.cc/jfgecz [following]
--2019-10-21 23:36:26--  https://tiny.cc/jfgecz
Connecting to tiny.cc (tiny.cc)|192.241.240.89|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.train.txt [following]
--2019-10-21 23:36:26--  https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59351 (58K) [text/plain]
Saving to:

In [0]:
!wget -O ukr_rus.test.txt http://tiny.cc/6zoeez

--2019-10-21 23:36:39--  http://tiny.cc/6zoeez
Resolving tiny.cc (tiny.cc)... 192.241.240.89
Connecting to tiny.cc (tiny.cc)|192.241.240.89|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://tiny.cc/6zoeez [following]
--2019-10-21 23:36:39--  https://tiny.cc/6zoeez
Connecting to tiny.cc (tiny.cc)|192.241.240.89|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.test.txt [following]
--2019-10-21 23:36:39--  https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12188 (12K) [text/plain]
Saving to: ‘

In [0]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")

In [0]:
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

## Embedding space mapping (0.3 pts)

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
or
$$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ - Frobenius norm.

$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit). So let's code.

In [0]:
from sklearn.linear_model import LinearRegression

# YOUR CODE HERE

mapping = LinearRegression(fit_intercept = False)

In [0]:
# fit our model
mapping.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None, normalize=False)

Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [0]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('апрель', 0.8531433343887329),
 ('июнь', 0.8402522802352905),
 ('март', 0.8385884761810303),
 ('сентябрь', 0.8331484198570251),
 ('февраль', 0.8311208486557007),
 ('октябрь', 0.8278019428253174),
 ('ноябрь', 0.8243728280067444),
 ('июль', 0.8229618072509766),
 ('август', 0.8112279772758484),
 ('январь', 0.8022986650466919)]

We can see that neighbourhood of this embedding cosists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed Ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in Russian embedding space).

In [0]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        # YOUR CODE HERE
        mapped_vec = mapped_vectors[i].reshape(1, -1)
        ru_word = pairs[i][1]
        uk_word = pairs[i][0]
        current_neighbours = ru_emb.most_similar(mapped_vec, topn = topn)
        
        word_in_neigh = False
        for neigh in np.array(current_neighbours)[:, 0]:
          # if word is in list
          if (ru_word == neigh):
            word_in_neigh = True
        
        if (word_in_neigh):
            num_matches += 1
        
    precision_val = num_matches / len(pairs)
    return precision_val

In [0]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
print(precision_top1)
print(precision_top5)

0.628498727735369
0.7913486005089059


## Making it better (orthogonal Procrustean problem) (0.3 pts)

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$W^*=UV^T$$

In [0]:
import numpy as np

In [0]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    # YOUR CODE GOES HERE
    # compute orthogonal embedding space mapping
    
    prod = np.dot(X_train.T, Y_train)
    U, Sigma, V_T = np.linalg.svd(prod)
    mapping = np.dot(U, V_T)

    return mapping

In [0]:
W = learn_transform(X_train, Y_train)

In [0]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('апрель', 0.8245131373405457),
 ('июнь', 0.8056631088256836),
 ('сентябрь', 0.8055763244628906),
 ('март', 0.8032934069633484),
 ('октябрь', 0.798710286617279),
 ('июль', 0.7946796417236328),
 ('ноябрь', 0.7939636707305908),
 ('август', 0.7938191294670105),
 ('февраль', 0.7923860549926758),
 ('декабрь', 0.7715376615524292)]

In [0]:
print(precision(uk_ru_test, np.matmul(X_test, W)))
print(precision(uk_ru_test, np.matmul(X_test, W), 5))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.6437659033078881
0.7989821882951654


## Unsupervised embedding-based MT (0.4 pts)

Now, let's build our word embeddings-based translator!

Firstly, download OPUS Tatoeba corpus.

In [0]:
!wget https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz

--2019-10-21 23:49:35--  https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1819128 (1.7M) [application/gzip]
Saving to: ‘uk.txt.gz’

uk.txt.gz           100%[===================>]   1.73M  2.26MB/s    in 0.8s    

2019-10-21 23:49:37 (2.26 MB/s) - ‘uk.txt.gz’ saved [1819128/1819128]



In [0]:
!gzip -d ./uk.txt.gz

gzip: ./uk.txt already exists; do you wish to overwrite (y or n)? y


In [0]:
with open('./uk.txt', 'r') as f:
    uk_corpus = f.readlines()

In [0]:
# To save your time and CPU, feel free to use first 1000 sentences of the corpus
uk_corpus = uk_corpus[:1000]

Приводим к нижнему регистру, чтобы корректно обращаться к словарю (он может и не содежать слова из начала предложения, находящегося в верхнем регистре, так что можем ошибочно сказать, что такого слова в словаре нет).


In [0]:
# Any necessary preprocessing if needed
# YOUR CODE HERE

#new_uk_corpus = []
# I will turn the letters to lower case
#for sentence in uk_corpus:
  #sentence = sentence.lower() 
#  new_uk_corpus.append(sentence.lower())

#uk_corpus = new_uk_corpus

В клетке выше закомментирована последняя строка (как раз перевод к нижнему регистру), так как в ходе работы выяснилось, что переход к нижнему регистру ухудшает качество перевода. Например, "Я" в начале предложений переводится как "мной", да и более значительные ошибки имеются. Например в данном списке предложений (см. ниже) "Александр" с заменой на "александр" будет переведен как "борис", что серьезно искажает смысл. Таким образом, обойдемся без препроцессинга.

In [0]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    # YOUR CODE GOES HERE
 
    # the vocab that we have: beyond this list we don't know any translations of uk-words
    uk_voc_exist = uk_emb.vocab.keys()
    
    translated = []
    words = sentence.split()
    for word in words:
      
      #And I also skip this "lower" for the same reasons
      #word = one_word.lower()
        if (word not in uk_voc_exist):
        #we add the natural(uk) form of word to save a little meaning
            translated.append(word)
        else:
            vect = uk_emb.get_vector(word)
            #ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])
            uk_emb_vect = [np.matmul(vect, W)]
        
            # 1 here is because we find the nearest word
            ru_translation = ru_emb.most_similar(uk_emb_vect, topn = 1)
            #print(ru_translation)
            ru_translation = ru_translation[0][0]
            translated.append(ru_translation)
    
    return " ".join(translated)

In [0]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Now you can play with your model and try to get as accurate translations as possible. **Note**: one big issue is out-of-vocabulary words. Try to think of various ways of handling it (you can start with translating each of them to a special **UNK** token and then move to more sophisticated approaches). Good luck!

In [0]:
for sent in uk_corpus[::10]:
    print(translate(sent))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Я уже закончу коледж, когда мы прибежишь со океании
Город бомбили враждебные літаки.
Можливо, мной антисоціальний, конечно это не означає, что мной не общаюсь со людьми
Впрочем утра выпала роса.
Беда не приходит одна
Посмотри по тот жеребей
Я заказал два гамбургера.
Я не хотел никого образити.
Гора покрыта снігом.
по фотографии во девушки корона не со золота, а со цветы
Во меня То мрія.
Я приехал во Японию со Китая
по север находится Шотландія; по юге — Англія; по востоке — Уельс; и ещe дальше по востоке — северная Ірландія.
Его родная страна — Німеччина.
Берн — столица Швейцарії.
Он ждал по него к десятой часа
Ты можешь взять ту книгу даром.
Такой роман сочинил известный американский писатель
Забронюйте, будте ласкаві, комнату возле международного аэропорта во Торонто.
Он знає, что ты его кохаєш?
Я знаю, что ты багатий.
Ті, кто всё забувають, щасливі.
Во этой реке опасно плавати.
Прийшов, побачив, переміг.
Я хожу к школы пішки.
Не моя справа!
Не забудь квиток.
Кто він?
Вы будете чай л

In [0]:
# Второй способ для неизвестных слов - просто заменять их на UNK-token
# Однако он несет меньше смысла, чем первый (оставлять слово из украинского словаря), 
# потому как в первом мы еще могли догадываться о значении слова, теперь же вся информация о нем пропадет

def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    # YOUR CODE GOES HERE
 
    # the vocab that we have: beyond this list we don't know any translations of uk-words
    uk_voc_exist = uk_emb.vocab.keys()
    
    translated = []
    words = sentence.split()
    for word in words:
      #word = one_word.lower()
        if (word not in uk_voc_exist):
            #we add the natural(uk) form of word to save a little meaning
            #translated.append(word)
            translated.append("<UNK>")
        else:
            vect = uk_emb.get_vector(word)
            #ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])
            uk_emb_vect = [np.matmul(vect, W)]
        
            # 1 here is because we find the nearest word
            ru_translation = ru_emb.most_similar(uk_emb_vect, topn = 1)
            #print(ru_translation)
            ru_translation = ru_translation[0][0]
            translated.append(ru_translation)
    
    return " ".join(translated)

In [0]:
# CHECK
for sent in uk_corpus[::10]:
    print(translate(sent))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Я уже закончу <UNK> когда мы прибежишь со океании
Город бомбили враждебные <UNK>
<UNK> мной <UNK> конечно это не <UNK> что мной не общаюсь со людьми
Впрочем утра выпала <UNK>
Беда не приходит одна
Посмотри по тот жеребей
Я заказал два <UNK>
Я не хотел никого <UNK>
Гора покрыта <UNK>
по фотографии во девушки корона не со <UNK> а со цветы
Во меня То <UNK>
Я приехал во Японию со Китая
по север находится <UNK> по юге — <UNK> по востоке — <UNK> и ещe дальше по востоке — северная <UNK>
Его родная страна — <UNK>
Берн — столица <UNK>
Он ждал по него к десятой часа
Ты можешь взять ту книгу <UNK>
Такой роман сочинил известный американский писатель
<UNK> будте <UNK> комнату возле международного аэропорта во <UNK>
Он <UNK> что ты его <UNK>
Я <UNK> что ты <UNK>
<UNK> кто всё <UNK> <UNK>
Во этой реке опасно <UNK>
<UNK> <UNK> <UNK>
Я хожу к школы <UNK>
Не моя <UNK>
Не забудь <UNK>
Кто <UNK>
Вы будете чай ли <UNK>
Он не пойдет по <UNK> как и мной
Когда Вы <UNK>
Это моя любимая песня
мы почти <UNK>
Как

Действительно, первый способ выглядит приятнее. Однако во втором сразу видно, где именно находятся незнакомые нам слова. Если бы возникла такая ситуация, что слово в русском и украинском языках пишется абсолютно одинаково, но при этом в словаре его нет, то в первом случае мы пропустили бы эту ситуацию, однако во втором получилось бы ее идентифицировать. Но в то же время в первом случае мы можем сами попытаться понять смысл неизвестных слов, чего не можем сделать во втором случае.

Про препроцессинг: если не делать преобразование к нижнему регистру, то некоторые слова переводятся красивее: например в начале предложений ставится не "мной", а "я", но в то же время существует опасность, что мы потеряем какой-нибудь перевод.

Great! 
See second notebook for the Neural Machine Translation assignment.